<h1>Flowmeter & Tera Term</h1>

Skripta za obradu podataka dobivenih iz Flowmetra preko Tere Terma u .txt formatu

<h2>Upišite naziv datoteke koju je potrebno obraditi:</h2>

In [26]:
ime_datoteke_bez_ext = 'ZW1 10.4'

<h2>Upišite željeni interval [u minutama] za izlazni excell dokument:</h2>

In [27]:
interval = 0.05

<h1>Pšvršina membrane u m2:</h1>

In [28]:
area = 0.046

<H1>[Ne diraj]</H1>

In [29]:
import pandas as pd
import re
import numpy as np
import seaborn as sb

ime_datoteke = ime_datoteke_bez_ext + '.txt'
#Broj sekunda između izlaznih rezultata (male poteškoće kod puno krivih rezultata 
#(čije je trajanje bilo veće od sekunde))
interval_u_sekundama = 60*interval  
data = []
errors =[]

with open(ime_datoteke) as f:
    read_data = f.read()
    data = read_data.split('\n')
    clean_data = []
    for line in data:
        m_data = re.findall(r".\d\d\d\.\d\d", line, flags=0)
        timestamp = re.findall(r"\d\d\:\d\d\:\d\d", line, flags=0)
        if len(m_data) ==6 and len(timestamp) == 1:
            flux = m_data[2]
            temp = m_data[1]
            press = m_data[0]
            time = timestamp[0]
            try:
                clean_data.append([pd.Timestamp(time), float(flux), float(press)*0.0689475729, float(temp)])
            except:
                pass


df = pd.DataFrame(clean_data, columns=['Time, min','Flow, mL/min','Pressure, bar', 'Temp., °C'])
df['Time, min'] = df['Time, min'] - df['Time, min'][0]
df['Time, min'] = df['Time, min'].dt.total_seconds()/60
df['Flux, L/m3h']= df['Flow, mL/min']*0.001*60/area
temp = df['Temp., °C']
df['Flux_cor_temp, L/m3h'] = df['Flux, L/m3h']*(1.79934+temp*(-0.04316)+temp*temp*0.000376037)
#df = df.set_index('Time, s').groupby(['Time, s']).mean().dropna(how='all')
df = df.groupby(['Time, min'],as_index=False).mean().dropna(how='all')
#
df = df.groupby(np.arange(len(df))//interval_u_sekundama).mean()
print(df)
 
writer = pd.ExcelWriter(ime_datoteke_bez_ext + '.xlsx')
df.to_excel(writer, 'Sheet1')
writer.save()



          Time, min  Flow, mL/min  Pressure, bar  Temp., °C  Flux, L/m3h  \
-0.0       0.016667      5.591407       0.051711  23.100000     7.293140   
 1.0       0.066667      5.594000       0.051711  23.100000     7.296522   
 2.0       0.116667      5.597000       0.051872  23.100000     7.300435   
 3.0       0.166667      2.673551       0.066932  23.100000     3.487240   
 4.0       0.216667    -41.490333       0.411479  23.100833   -54.117826   
 5.0       0.266667     -7.304000       0.173932  23.117833    -9.526957   
 6.0       0.316667     35.917649      -0.046180  23.120000    46.849108   
 7.0       0.366667     37.594667      -0.054721  23.113500    49.036522   
 8.0       0.416667     38.073333      -0.055859  23.106167    49.660870   
 9.0       0.466667     37.400888      -0.055850  23.097059    48.783768   
 10.0      0.516667     38.154333      -0.056200  23.082667    49.766522   
 11.0      0.566667     38.466560      -0.057455  23.059325    50.173774   
 12.0      0